In [1]:
import pandas as pd
from pathlib import Path

In [30]:
path_to_data = Path('../california/train/')
# files need to be in order they are expected by combine
necessary_files = ['X02_RACE', 'X01_AGE_AND_SEX', 'X15_EDUCATIONAL_ATTAINMENT',
                  'X12_MARITAL_STATUS_AND_HISTORY', 'X19_INCOME', 'X03_HISPANIC_OR_LATINO_ORIGIN']

In [3]:
def raw_to_clean_education(Raw) :
    educationAttainment = pd.read_csv(Raw)
    clean_educationAttainment = educationAttainment[['B15003e1']].copy() #Total pop
    
    clean_educationAttainment['column1'] = educationAttainment[['B15003e2']].agg("sum", axis="columns")
    clean_educationAttainment['column2'] = educationAttainment[['B15003e3','B15003e4','B15003e5','B15003e6','B15003e7','B15003e8']].agg("sum", axis="columns")
    clean_educationAttainment['column3'] = educationAttainment[['B15003e9','B15003e10','B15003e11','B15003e12']].agg("sum", axis="columns")
    clean_educationAttainment['column4'] = educationAttainment[['B15003e13','B15003e14','B15003e15','B15003e16']].agg("sum", axis="columns")
    clean_educationAttainment['column5'] = educationAttainment[['B15003e17','B15003e18']].agg("sum", axis="columns")
    clean_educationAttainment['column6'] = educationAttainment[['B15003e19','B15003e20']].agg("sum", axis="columns")
    clean_educationAttainment['column7'] = educationAttainment[['B15003e21']].agg("sum", axis="columns")
    clean_educationAttainment['column8'] = educationAttainment[['B15003e22']].agg("sum", axis="columns")
    clean_educationAttainment['column9'] = educationAttainment[['B15003e23']].agg("sum", axis="columns")
    clean_educationAttainment['column10'] = educationAttainment[['B15003e24']].agg("sum", axis="columns")
    clean_educationAttainment['column11'] = educationAttainment[['B15003e25']].agg("sum", axis="columns")
    
    clean_educationAttainment['No Schooling'] = clean_educationAttainment.apply(lambda x: x['column1'] if x['column1'] < 1 else x['column1']/x['B15003e1'], axis=1)
    clean_educationAttainment['Primary School'] = clean_educationAttainment.apply(lambda x: x['column2'] if x['column2'] < 1 else x['column2']/x['B15003e1'], axis=1)
    clean_educationAttainment['Middle School'] = clean_educationAttainment.apply(lambda x: x['column3'] if x['column3'] < 1 else x['column3']/x['B15003e1'], axis=1)
    clean_educationAttainment['High School (No Diploma)'] = clean_educationAttainment.apply(lambda x: x['column4'] if x['column4'] < 1 else x['column4']/x['B15003e1'], axis=1)
    clean_educationAttainment['High School Diploma or Equivalent'] = clean_educationAttainment.apply(lambda x: x['column5'] if x['column5'] < 1 else x['column5']/x['B15003e1'], axis=1)
    clean_educationAttainment['College (No Degree)'] = clean_educationAttainment.apply(lambda x: x['column6'] if x['column6'] < 1 else x['column6']/x['B15003e1'], axis=1)
    clean_educationAttainment['Associate Degree'] = clean_educationAttainment.apply(lambda x: x['column7'] if x['column7'] < 1 else x['column7']/x['B15003e1'], axis=1)
    clean_educationAttainment['Bachelors Degree'] = clean_educationAttainment.apply(lambda x: x['column8'] if x['column8'] < 1 else x['column8']/x['B15003e1'], axis=1)
    clean_educationAttainment['Masters Degree'] = clean_educationAttainment.apply(lambda x: x['column9'] if x['column9'] < 1 else x['column9']/x['B15003e1'], axis=1)
    clean_educationAttainment['Professional School Degree'] = clean_educationAttainment.apply(lambda x: x['column10'] if x['column10'] < 1 else x['column10']/x['B15003e1'], axis=1)
    clean_educationAttainment['Doctorate Degree'] = clean_educationAttainment.apply(lambda x: x['column11'] if x['column11'] < 1 else x['column11']/x['B15003e1'], axis=1)
    
    # Drop all unnecessary columns
    clean_educationAttainment = clean_educationAttainment.drop(['B15003e1'],axis=1)
    clean_educationAttainment = clean_educationAttainment.drop(['column1','column2','column3','column4','column5','column6','column7','column8','column9','column10','column11'],axis=1)
    
    # Add GEOID and ensure it is in the first column
    clean_educationAttainment['GEOID'] = educationAttainment['GEOID']
    cols = clean_educationAttainment.columns.tolist()
    cols.insert(0, cols.pop(cols.index('GEOID')))
    clean_educationAttainment = clean_educationAttainment.reindex(columns=cols)
    
    return clean_educationAttainment

In [4]:
def raw_to_clean_gender(Raw) :
    agesex = pd.read_csv(Raw)
    clean_agesex = agesex[['B01001e1','B01001e2','B01001e26']].copy() #Total pop, total male, total female
    
    # Find proportion of males and females
    clean_agesex['Male'] = clean_agesex.apply(lambda x: x['B01001e2'] if x['B01001e2'] < 1 else x['B01001e2']/x['B01001e1'], axis=1)
    clean_agesex['Female'] = clean_agesex.apply(lambda x: x['B01001e26'] if x['B01001e26'] < 1 else x['B01001e26']/x['B01001e1'], axis=1)
    
    # Aggregate age groups together, regardless of gender
    clean_agesex['age1'] = agesex[['B01001e7','B01001e8','B01001e9','B01001e10','B01001e31','B01001e32','B01001e33','B01001e34']].agg("sum", axis="columns")
    clean_agesex['age2'] = agesex[['B01001e11','B01001e12','B01001e35','B01001e36']].agg("sum", axis="columns")
    clean_agesex['age3'] = agesex[['B01001e13','B01001e14','B01001e37','B01001e38']].agg("sum", axis="columns")
    clean_agesex['age4'] = agesex[['B01001e15','B01001e16','B01001e39','B01001e40']].agg("sum", axis="columns")
    clean_agesex['age5'] = agesex[['B01001e17','B01001e18','B01001e19','B01001e41','B01001e42','B01001e43']].agg("sum", axis="columns")
    
    # Find proportion of each age range
    clean_agesex['18-24'] = clean_agesex.apply(lambda x: x['age1'] if x['age1'] < 1 else x['age1']/x['B01001e1'], axis=1)
    clean_agesex['25-34'] = clean_agesex.apply(lambda x: x['age2'] if x['age2'] < 1 else x['age2']/x['B01001e1'], axis=1)
    clean_agesex['35-44'] = clean_agesex.apply(lambda x: x['age3'] if x['age3'] < 1 else x['age3']/x['B01001e1'], axis=1)
    clean_agesex['45-54'] = clean_agesex.apply(lambda x: x['age4'] if x['age4'] < 1 else x['age4']/x['B01001e1'], axis=1)
    clean_agesex['55-64'] = clean_agesex.apply(lambda x: x['age5'] if x['age5'] < 1 else x['age5']/x['B01001e1'], axis=1)
    
    # Drop all unnecessary columns
    clean_agesex = clean_agesex.drop(['B01001e1','B01001e2','B01001e26'],axis=1)
    clean_agesex = clean_agesex.drop(['age1','age2','age3','age4','age5'],axis=1)
    
    # Add GEOID and ensure it is in the first column
    clean_agesex['GEOID'] = agesex['GEOID']
    cols = clean_agesex.columns.tolist()
    cols.insert(0, cols.pop(cols.index('GEOID')))
    clean_agesex = clean_agesex.reindex(columns= cols)
    
    return clean_agesex

In [5]:
def raw_to_clean_marital(Raw) :
    marital = pd.read_csv(Raw)
    clean_marital = marital[['B12001e1']].copy() #Total pop
    
    # Categories
    clean_marital['marriedP'] = marital[['B12001e5','B12001e14']].agg("sum", axis="columns") #married present
    clean_marital['marriedA'] = marital[['B12001e6','B12001e15']].agg("sum", axis="columns") #married absent
    clean_marital['NM'] = marital[['B12001e3','B12001e12']].agg("sum", axis="columns") #never married
    clean_marital['D'] = marital[['B12001e10','B12001e19']].agg("sum", axis="columns") #divorced
    clean_marital['W'] = marital[['B12001e9','B12001e18']].agg("sum", axis="columns") #widowed
    
#     # Female Categories
#     clean_marital['marriedPf'] = marital['B12001e14'] #married present
#     clean_marital['marriedAf'] = marital['B12001e15'] #married absent
#     clean_marital['NMf'] = marital['B12001e12'] #never married
#     clean_marital['Df'] = marital['B12001e19'] #divorced
#     clean_marital['Wf'] = marital['B12001e18'] #widowed
    
    # Find proportions for males
    clean_marital['Married Present'] = clean_marital.apply(lambda x: x['marriedP'] if x['marriedP'] < 1 else x['marriedP']/x['B12001e1'], axis=1)
    clean_marital['Married Absent'] = clean_marital.apply(lambda x: x['marriedA'] if x['marriedA'] < 1 else x['marriedA']/x['B12001e1'], axis=1)
    clean_marital['Never Married'] = clean_marital.apply(lambda x: x['NM'] if x['NM'] < 1 else x['NM']/x['B12001e1'], axis=1)
    clean_marital['Divorced'] = clean_marital.apply(lambda x: x['D'] if x['D'] < 1 else x['D']/x['B12001e1'], axis=1)
    clean_marital['Widowed'] = clean_marital.apply(lambda x: x['W'] if x['W'] < 1 else x['W']/x['B12001e1'], axis=1)
    
#     # Find proportions for females
#     clean_marital['Female: Married Present'] = clean_marital.apply(lambda x: x['marriedPf'] if x['marriedPf'] < 1 else x['marriedPf']/x['B12001e11'], axis=1)
#     clean_marital['Female: Married Absent'] = clean_marital.apply(lambda x: x['marriedAf'] if x['marriedAf'] < 1 else x['marriedAf']/x['B12001e11'], axis=1)
#     clean_marital['Female: Never Married'] = clean_marital.apply(lambda x: x['NMf'] if x['NMf'] < 1 else x['NMf']/x['B12001e11'], axis=1)
#     clean_marital['Female: Divorced'] = clean_marital.apply(lambda x: x['Df'] if x['Df'] < 1 else x['Df']/x['B12001e11'], axis=1)
#     clean_marital['Female: Widowed'] = clean_marital.apply(lambda x: x['Wf'] if x['Wf'] < 1 else x['Wf']/x['B12001e11'], axis=1)

    # Drop all unnecessary columns
    clean_marital = clean_marital.drop(['B12001e1'],axis=1)
    clean_marital = clean_marital.drop(['marriedP','marriedA','NM','D','W'],axis=1)
#     clean_marital = clean_marital.drop(['marriedPf','marriedAf','NMf','Df','Wf'],axis=1)
    
    # Add GEOID and ensure it is in the first column
    clean_marital['GEOID'] = marital['GEOID']
    cols = clean_marital.columns.tolist()
    cols.insert(0, cols.pop(cols.index('GEOID')))
    clean_marital = clean_marital.reindex(columns= cols)
    
    return clean_marital

In [31]:
def preprocess_race(csv_path):
    SELECTED_COL_SHORTNAME = ['GEOID', 'B02001e1', 'B02001e2', 'B02001e3', 'B02001e4',\
                              'B02001e5', 'B02001e6', 'B02001e7', 'B02001e8']
    OUTPUT_COL_NAME = [ "geoid", "race_total", "race_white","race_black", "race_american-indian",\
                        "race_asian", "race_native-hawaiian","race_other", "race_multiracial"]
    race_data = pd.read_csv(csv_path)
    race_data = race_data[SELECTED_COL_SHORTNAME]
    # Form output data:
    data_out = pd.DataFrame()
    data_out = race_data
    data_out.columns = OUTPUT_COL_NAME
    for idx in range(2, len(SELECTED_COL_SHORTNAME)):
        data_out[OUTPUT_COL_NAME[idx]] = data_out[OUTPUT_COL_NAME[idx]] / data_out[OUTPUT_COL_NAME[1]]    
    # Drop race_total:
    data_out = data_out.drop("race_total", axis=1) 
    # Drop nan values:
    data_out = data_out.dropna()
    return data_out

In [32]:
def preprocess_hispanic(csv_path):
    hispanic = pd.read_csv(csv_path)
    hisp_count = hispanic['B03002e1']
    non_hisp_count = hispanic['B03002e2']
    out = pd.DataFrame()
    out['GEOID'] = hispanic['GEOID']
    out['hispanic_percentage'] = hisp_count / (hisp_count + non_hisp_count)
    out = out.dropna()
    return out
    

In [37]:
def combine(race_csv,gender_csv,education_csv,marital_csv,income_csv, hispanic_csv) :
    '''
    args: path to relevant csv file
    returns: merged pd.DataFrame
    '''
    race = preprocess_race(race_csv).rename(index=str,columns={'geoid':'GEOID'})
    gender = raw_to_clean_gender(gender_csv)
    education = raw_to_clean_education(education_csv)
    marital = raw_to_clean_marital(marital_csv)
    hispanic = preprocess_hispanic(hispanic_csv)
    df1 = pd.merge(race,gender,on='GEOID')
    df2 = pd.merge(df1,education,on='GEOID')
    df3 = pd.merge(df2,marital,on='GEOID')
    df4 = pd.merge(df3, hispanic, on='GEOID')
    return df4


# combined = combine(*[path_to_data / f'{f}.csv' for f in necessary_files])
combined.to_csv('Combined.csv')